In [19]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import re

In [16]:
def get_service_ticket(api_key, service="http://umlsks.nlm.nih.gov"):
    tgt_resp = requests.post(
        "https://utslogin.nlm.nih.gov/cas/v1/api-key",
        data={"apikey": api_key},
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    tgt_match = re.search(r'action="(.+?)"', tgt_resp.text)
    if not tgt_match:
        raise ValueError("TGT 获取失败")
    tgt_url = tgt_match.group(1)
    st_resp = requests.post(tgt_url, data={"service": service})
    return st_resp.text

In [24]:
api_key = os.getenv("NX_API_KEY")
print("API Key:", api_key)

API Key: None


In [20]:
api_key = os.getenv("NX_API_KEY")
ticket = get_service_ticket(api_key)
url = f"https://uts-ws.nlm.nih.gov/rest/content/current/CUI/C0004057?ticket={ticket}"
response = requests.get(url)
print(response.json())

ValueError: TGT 获取失败

In [ ]:
tgt_url = "https://utslogin.nlm.nih.gov/cas/v1/api-key"

tgt_response = requests.post(
    tgt_url,
    data={"apikey": api_key},
    headers={"Content-Type": "application/x-www-form-urlencoded"}
)

# 从返回的 HTML 中提取 TGT url（form action）
import re
match = re.search(r'action="(.+?)"', tgt_response.text)
TGT = match.group(1) if match else None
print("TGT:", TGT)


TGT: None


In [14]:
import pandas as pd

columns = [
    "RXCUI", "LAT", "TS", "LUI", "STT", "SUI", "ISPREF", "RXAUI", "SAUI", "SCUI", "SDUI",
    "SAB", "TTY", "CODE", "STR", "SRL", "SUPPRESS", "CVF"
]

RXNCONSO = pd.read_csv(
    '/Users/dylanli/Downloads/RxNorm_weekly_prescribe_07162025/rrf/RXNCONSO.RRF',
    sep='|',
    header=None,
    names=columns,
    usecols=["RXCUI", "STR", "TTY", "CODE", "SAB"],  # 只读取前 19 列，忽略最后一个空字段
    engine='python',              # 使用 python 引擎，能更宽容处理分隔符
    dtype=str
)

print(RXNCONSO.head())

ValueError: Number of passed names did not match number of header fields in the file

In [1]:
import pandas as pd

# Step 1: 加载 CSV 文件
conso_cols = [
    "RXCUI", "LAT", "TS", "LUI", "STT", "SUI", "ISPREF", "RXAUI", "SAUI", "SCUI", "SDUI",
    "SAB", "TTY", "CODE", "STR", "SRL", "SUPPRESS", "CVF"
]
sat_cols = [
    "RXCUI", "LUI", "SUI", "RXAUI", "STYPE", "CODE", "ATUI", "SATUI",
    "ATN", "SAB", "ATV", "SUPPRESS", "CVF"
]

rxnconso = pd.read_csv("/Users/dylanli/Downloads/RxNorm_weekly_prescribe_07162025/rrf/RXNCONSO.RRF", dtype=str, usecols=conso_cols)
rxnsat = pd.read_csv("/Users/dylanli/Downloads/RxNorm_weekly_prescribe_07162025/rrf/RXNSAT.RRF", dtype=str, usecols=sat_cols)

# Step 2: 过滤只保留英文、首选名称的条目
rxnconso = rxnconso[(rxnconso["LAT"] == "ENG") & (rxnconso["ISPREF"] == "Y")]
rxnconso = rxnconso[["RXCUI", "STR", "TTY", "CODE", "SAB"]].drop_duplicates()

# Step 3: 筛选属性说明（说明性文字）
# 如果你希望放宽条件，注释掉下一行
rxnsat = rxnsat[rxnsat["ATN"].str.lower().str.contains("description|display|text", na=False)]
rxnsat = rxnsat[["RXCUI", "ATN", "ATV"]].drop_duplicates()

# Step 4: 合并两个表
kb = pd.merge(rxnconso, rxnsat, on="RXCUI", how="left")

# Step 5: 每个药品一个条目（group by RXCUI）
kb_final = kb.groupby("RXCUI").agg({
    "STR": "first",  # 药品名
    "TTY": "first",  # 术语类型
    "CODE": "first", # 源代码
    "SAB": "first",  # 数据源
    "ATV": lambda x: " | ".join(x.dropna().unique())  # 多个描述合并
}).reset_index()

# Step 6: 保存输出
kb_final.to_csv("Database/rxnorm_knowledge_base.csv", index=False)
print("Medical Knowledge base completed as rxnorm_knowledge_base.csv")


ValueError: Usecols do not match columns, columns expected but not found: ['CODE', 'SAUI', 'SRL', 'CVF', 'SUPPRESS', 'RXAUI', 'LUI', 'LAT', 'ISPREF', 'SCUI', 'SDUI', 'SAB', 'STR', 'TTY', 'SUI', 'TS', 'RXCUI', 'STT']